In [16]:
import pandas as pd

In [17]:
RecList = pd.read_csv('../CSV/recipeList.csv')

In [18]:
ioDict = {}

for row in RecList.iterrows():    
    if row[1][1] in ioDict:
        ioDict[row[1][1]] += 1
    else:
        ioDict[row[1][1]] = 0

In [19]:
maxVal = 0
for key, val in ioDict.items():
    if val > maxVal:
        maxVal = val
    else:
        continue

In [20]:
for key, val in ioDict.items():
    ioDict[key] = val/maxVal

In [27]:
ingPairs = []
keyList = list(ioDict.keys())

for key1idx in range(0, len(keyList)):
    for key2idx in range((key1idx + 1), len(keyList)):
        temp = []
        temp.append(keyList[key1idx])
        temp.append(keyList[key2idx])
        ingPairs.append(temp)

del temp
del keyList

In [28]:
ipWeights = [0] * len(ingPairs)

In [29]:
curRecipe = ''
ingList = []
for row in RecList.iterrows():
    if curRecipe == row[1][0]:
        ingList.append(row[1][1])
    else:
        curRecipe = row[1][0]
        for pairidx in range(0, len(ingPairs)):
            if ingPairs[pairidx][0] in ingList and ingPairs[pairidx][1] in ingList:
                ipWeights[pairidx] += 1
        ingList = []

del ingList

In [32]:
print(len(ipWeights), len(ingPairs))

43956 43956


In [33]:
normIPweights = [0] * len(ipWeights)

In [34]:
maxWeight = max(ipWeights)

for idx in range(0, len(ipWeights)):
    normIPweights[idx] = ipWeights[idx]/maxWeight

In [37]:
firstIng = []
secondIng = []

for idx in range(0, len(ingPairs)):
    firstIng.append(ingPairs[idx][0])
    secondIng.append(ingPairs[idx][1])

In [38]:
ingPairWeights = pd.DataFrame(list(zip(firstIng, secondIng, normIPweights)), columns=['First Ingredient', 'Second Ingredient', 'Normalised Weight'])

In [ ]:
ingPairWeights.to_csv("../CSV/ingPairWeights.csv", index = False)